In [1]:
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay
from dateutil.relativedelta import relativedelta
from collections import defaultdict 
import pprint

from commons import read_xls, get_stock_data, save_csv, get_holding_quantities
from commons import min_consecutive_sum_kadane, get_curr_prices_from_holdings
from consts import HOLDING_FILE

In [2]:
# HOLDINGS_FILE_NAME = '../docs/Holdings_28Dec23.xlsx'

In [3]:
def process_sheets(df, today=None):
    if not today:
        today = datetime.datetime.today()
        
    start = today - BDay(1) # Get prev business date as market didnt start today
    end = today

#     if datetime.time(today.hour, today.minute, today.second) < datetime.time(9, 15, 5):
#         print('Markets didnt start yet, getting data from prev business dates')
#         end = start
#         start -= BDay(2)
    
    try:
        print(f'****** Calling for stock data between {start} and {end} *******')

        curr_prices = get_stock_data(start=start, end=end, print_data=False)
        curr_prices = curr_prices.reset_index().transpose().reset_index().rename(columns={0: 'currPrice'}) #.rename(columns={'index': 'Symbol'})
    except e: 
        print(e)
        print('***** reading current prices from Holdings.txt as yahoo crapped again!!!')
        curr_prices = get_curr_prices_from_holdings()
        
    df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
    df = df.drop('index', axis=1)
    df = df.sort_values(by='Date', ascending=True)
    df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']

#     def flip_sells(t, val):
#         if t == 'sell':
#             val *= -1
            
#         return val

#     df['PnL']      = df.apply(lambda x: flip_sells(x['Trade Type'], x['PnL']), axis=1)
#     df['Quantity'] = df.apply(lambda x: flip_sells(x['Trade Type'], x['Quantity']), axis=1)
    return df

def get_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
#     df = df[df['Trade Date'] > str(date)] # Get trades after this date
    df = df.sort_values(by='Date', ascending=True)
    data = {}
    
    for c in get_holding_quantities().keys():
        df_c = df[df['COMPANY'] == c]
        df_dict = df_c.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
        ltcg, stcg, ltcg_qty, stcg_qty = 0, 0, 0, 0
        data[c] = defaultdict(int)
        
        for i, (comp, dt, bp, qty, cp, pnl) in enumerate(zip(df_dict['COMPANY'], df_dict['Date'], df_dict['BuyPrice'], \
                                             df_dict['Qty'], df_dict['currPrice'], df_dict['PnL'])):
            i += 1 
            
            if dt.date() <= stcg_cutoff_date:
                ### LTCG
#                 print(f'LTCG {i} {dt.date()} >= {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                data[c]['ltcg_qty'] += qty
                data[c]['ltcg'] += pnl
            else:
                ### STCG
#                 print(f'STCG {i} {dt.date()} <  {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                
                # STCLoss only if prev last LTCG txn was negative
                if 'stcg_qty' not in data[c]: # init stcg total qty 
                    data[c]['stcl_qty'] = data[c]['ltcg_qty']
                    data[c]['stcg_qty'] = data[c]['ltcg_qty']

                if pnl < 0: ## only if it is a loss
                    data[c]['stcl_qty'] += qty
                    data[c]['stcl'] += pnl
                else:
                    data[c]['stcg_qty'] += qty
                    data[c]['stcg'] += pnl                    
                
            data[c]['total_qty'] += qty
            data[c]['total'] += pnl            

        
        
        
#     for s in QTYS.keys():
#         s_df = df[df['Symbol'] == s].sort_values(by='Trade Date', ascending=True)
#         print(s_df.to_dict())
#         break
    
    return data

def get_max_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
    data = {}
    
    for c in get_holding_quantities().keys():
        df_c = df[(df['Date'] > str(stcg_cutoff_date)) & (df['COMPANY'] == c)] # Get trades after this date
        df_dict = df_c.to_dict('list')
        data[c] = min_consecutive_sum_kadane(df_dict['PnL'], df_dict['Date'])

    return {k: v for k, v in sorted(data.items(), key=lambda item: item[1])} # sort data by value before returning



In [8]:
df = read_xls(HOLDING_FILE, tab=5)
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]

df = process_sheets(df)
# save_csv(df, 'ltcg_stcg')

print('******* Max STCL - longest STCL loss streak *******')
print(pd.DataFrame(get_max_stcl(df)).T)
pd.DataFrame(get_stcl(df)).T.sort_values(by='stcl', ascending=True)

****** Calling for stock data between 2024-01-23 04:01:19.405361 and 2024-01-24 04:01:19.405361 *******
****** Getting stock data between 2024-01-23 04:01:19.405361 and 2024-01-24 04:01:19.405361 *******
[*********************100%%**********************]  24 of 24 completed


3 Failed downloads:
['RAJGLOWIR.BO', 'MOLDTEK.BO', 'DEEPAKNI.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-01-23 04:01:19.405361 -> 2024-01-24 04:01:19.405361)')



****** Got stock data between 2024-01-23 04:01:19.405361 and 2024-01-24 04:01:19.405361 *******
******* Max STCL - longest STCL loss streak *******
                      0          1          2
BAJFINANCE -7042.293945  11Jul2023  23Jan2024
FINEORG     -6893.85498  22Jun2023  20Jan2024
HAPPSTMNDS -4176.598145  23Jun2023  28Sep2023
DMART      -2596.901855  19Oct2023  20Jan2024
BERGEPAINT      -2110.0  28Sep2023  20Jan2024
SONACOMS    -1229.10083  28Dec2023  28Dec2023
DEVYANI     -958.800156  21Aug2023  21Aug2023
IONEXCHANG  -676.499634  28Dec2023  28Dec2023
PRINCEPIPE  -361.999512  03Oct2023  03Oct2023
DIVISLAB    -240.349365  17Aug2023  25Aug2023
TATACHEM     -83.400146  04Oct2023  04Oct2023
KPITTECH         -26.25  28Dec2023  28Dec2023
NAUKRI                0          1          0
AFFLE                 0          1          0
DIXON                 0          1          0
RELAXO                0          1          0
LAURUSLABS            0          1          0
PRAJIND               0

,ltcg_qty,ltcg,total_qty,total,stcl_qty,stcg_qty,stcg,stcl
FINEORG,0.0,NaN,59.0,-5647.655762,42.0,17.0,2948.048340,-8595.704102
BAJFINANCE,18.0,13041.603516,55.0,11883.510742,43.0,30.0,6566.402344,-7724.495117
HAPPSTMNDS,116.0,-18027.047168,245.0,-21524.694019,212.0,149.0,695.950806,-4193.597656
DMART,36.0,-18156.603516,93.0,-17748.309082,66.0,63.0,3385.297363,-2977.002930
BERGEPAINT,161.0,-16479.250000,298.0,-942.500000,241.0,218.0,17646.750000,-2110.000000
SONACOMS,234.0,-2341.105713,551.0,5675.336548,298.0,487.0,9310.043823,-1293.601562
DEVYANI,180.0,2650.999451,281.0,3052.199142,231.0,230.0,1359.999847,-958.800156
IONEXCHANG,220.0,70616.202686,273.0,71445.053333,250.0,243.0,1505.350281,-676.499634
PRINCEPIPE,128.0,-843.796875,213.0,3337.705200,148.0,193.0,4543.501587,-361.999512
DIVISLAB,32.0,-15735.398438,55.0,-12577.247314,45.0,42.0,3398.500488,-240.349365


In [5]:
df = read_xls('../docs/Holdings_16Dec23.xlsx', tab=5)
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]
df = process_sheets(df)
# df_dict = df.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
# df_dict
df

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


****** Calling for stock data between 2024-01-23 03:57:53.072350 and 2024-01-24 03:57:53.072350 *******
****** Getting stock data between 2024-01-23 03:57:53.072350 and 2024-01-24 03:57:53.072350 *******
[*********************100%%**********************]  24 of 24 completed


3 Failed downloads:
['RAJGLOWIR.BO', 'MOLDTEK.BO', 'DEEPAKNI.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-01-23 03:57:53.072350 -> 2024-01-24 03:57:53.072350)')



****** Got stock data between 2024-01-23 03:57:53.072350 and 2024-01-24 03:57:53.072350 *******


,COMPANY,Date,Qty,BuyPrice,currPrice,PnL
0,DIXON,2021-04-12,6,3580.0,5856.25,13657.5
5,NAUKRI,2021-04-16,4,4601.0,4943.549805,1370.199219
6,NAUKRI,2021-04-23,4,4870.0,4943.549805,294.199219
21,TATACHEM,2021-05-04,35,729.5,1004.049988,9609.249573
22,TATACHEM,2021-05-05,15,700.0,1004.049988,4560.749817
...,...,...,...,...,...,...
190,RAJRATAN,2023-11-22,5,740.0,NaN,NaN
329,FINEORG,2023-11-22,3,4240.0,4459.649902,658.949707
285,BAJFINANCE,2023-12-20,3,7501.0,7070.700195,-1290.899414
241,SONACOMS,2023-12-20,30,540.0,579.849976,1195.499268


In [6]:
# print(s_df)
# s_df.to_dict()
# sum(s_df['Quantity'])
s_df['PnL'] = (s_df['currPrice'] - s_df['Price']) * s_df['Quantity']
print(s_df)
print(sum(s_df[s_df['Trade Type'] == 'buy']['PnL']) - sum(s_df[s_df['Trade Type'] == 'sell']['PnL']))

def f(t, pnl):
    if t == 'sell':
        return -1*pnl
    return pnl

s_df['PnL'] = s_df.apply(lambda x: f(x['Trade Type'], x['PnL']), axis=1)
s_df
# s_df['Trade Type', 'PnL'].apply(lambda x, y: f(x, y))
# s_df[['Trade Type', 'PnL']]

NameError: name 's_df' is not defined

In [ ]:
datetime.datetime.today().minute
# datetime.datetime.today() - BDay(1)
t = datetime.time(9, 30, 59)
t.hour
# datetime.time(datetime.datetime.today())
datetime.time(datetime.datetime.today().hour, datetime.datetime.today().minute) < datetime.time(9, 15, 5)
datetime.datetime.today().second

today = datetime.datetime.today()
today - BDay(2), today - BDay(1) # Get prev business date as market didnt start today

In [ ]:
curr_prices = get_curr_prices_from_holdings()
# print(curr_prices)
df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
df = df.drop('index', axis=1)
df = df.sort_values(by='Date', ascending=True)
df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']
df